## Quickstart tutorial

In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Check CUDA and BF16 support

In [10]:
torch.cuda.is_available()

True

In [11]:
torch.cuda.is_bf16_supported()

True

Download the FashionMNIST dataset. The `transform` and `target_transform` arguments can be used to transform the features and labels, respectively. These are stored in a `Dataset` object.

In [15]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root=".data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root=".data",
    train=False,
    download=True,
    transform=ToTensor(),
)

A `DataLoader` class handles streaming, batching, sampling, and shuffling the data.

In [17]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [32]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

m = NeuralNetwork().to(device)
print(m)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [33]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(m.parameters(), lr=1e-3)

In [34]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [35]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [36]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, m, loss_fn, optimizer)
    test(test_dataloader, m, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309330  [   64/60000]
loss: 2.290163  [ 6464/60000]
loss: 2.265549  [12864/60000]
loss: 2.259950  [19264/60000]
loss: 2.241203  [25664/60000]
loss: 2.226794  [32064/60000]
loss: 2.227386  [38464/60000]
loss: 2.197522  [44864/60000]
loss: 2.188655  [51264/60000]
loss: 2.159012  [57664/60000]
Test Error: 
 Accuracy: 49.0%, Avg loss: 2.149080 

Epoch 2
-------------------------------
loss: 2.158190  [   64/60000]
loss: 2.145391  [ 6464/60000]
loss: 2.081802  [12864/60000]
loss: 2.108133  [19264/60000]
loss: 2.042045  [25664/60000]
loss: 1.995088  [32064/60000]
loss: 2.019863  [38464/60000]
loss: 1.937240  [44864/60000]
loss: 1.946060  [51264/60000]
loss: 1.872039  [57664/60000]
Test Error: 
 Accuracy: 52.5%, Avg loss: 1.867332 

Epoch 3
-------------------------------
loss: 1.894179  [   64/60000]
loss: 1.867814  [ 6464/60000]
loss: 1.742086  [12864/60000]
loss: 1.807457  [19264/60000]
loss: 1.673818  [25664/60000]
loss: 1.636036  [32064/600

Saving and loading models

In [39]:
torch.save(m.state_dict(), "model0.pth")

m1 = NeuralNetwork()
m1.load_state_dict(torch.load("model0.pth"))

<All keys matched successfully>

In [38]:
torch.load("model0.pth")

'm'

In [54]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

m0 = m.to("cpu")

m1.eval()
with torch.no_grad():
    for i in range(10):
        x, y = test_data[i][0], test_data[i][1]
        pred = m0(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        assert pred[0][0] == m(x)[0][0]
        print(f'{"+" if predicted == actual else "-"} Predicted: "{predicted}", Actual: "{actual}"')

+ Predicted: "Ankle boot", Actual: "Ankle boot"
+ Predicted: "Pullover", Actual: "Pullover"
+ Predicted: "Trouser", Actual: "Trouser"
+ Predicted: "Trouser", Actual: "Trouser"
- Predicted: "Pullover", Actual: "Shirt"
+ Predicted: "Trouser", Actual: "Trouser"
+ Predicted: "Coat", Actual: "Coat"
- Predicted: "Coat", Actual: "Shirt"
- Predicted: "Sneaker", Actual: "Sandal"
+ Predicted: "Sneaker", Actual: "Sneaker"
